In [1]:
import pandas as pd
from urbansim_templates.data import LoadTable
import orca

In [2]:
hdf_path = '../notebooks/custom_mpo_06197001_model_data.h5'
hdf_tables = ['households', 'persons']

for table_name in hdf_tables:
    step_name = 'load_' + table_name
    LoadTable(table=table_name, source_type='hdf', path = hdf_path,
              extra_settings={'key': table_name}, name=step_name).run()

In [3]:
persons = orca.get_table('persons').local
households = orca.get_table('households').local

In [4]:
persons

,age,earning,edu,hispanic,hours,member_id,race_id,relate,sex,student,work_at_home,worker,household_id,person_age,person_sex,race,hispanic.1,p_hispanic,MAR
person_id,,,,,,,,,,,,,,,,,,,
0,63,0.0,18.0,0,0.0,1,2,0,2,0,0,0,223,above 60,female,black,0,no,5
1,63,0.0,18.0,0,0.0,1,2,0,2,0,0,0,1179,above 60,female,black,0,no,5
2,68,0.0,22.0,0,0.0,1,2,0,2,0,0,0,861,above 60,female,black,0,no,1
3,68,0.0,22.0,0,0.0,1,2,0,2,0,0,0,862,above 60,female,black,0,no,1
4,68,0.0,22.0,0,0.0,1,2,0,2,0,0,0,865,above 60,female,black,0,no,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6808832,24,50000.0,21.0,0,40.0,1,1,0,1,0,0,1,2607796,20 to 35,male,white,0,no,5
6808833,33,44000.0,16.0,0,40.0,1,1,0,1,0,0,1,2607818,20 to 35,male,white,0,no,5
6808834,33,44000.0,16.0,0,40.0,1,1,0,1,0,0,1,2607819,20 to 35,male,white,0,no,5


In [8]:
(~persons["household_id"].isin(households.index)).sum()

0

In [11]:
(1 - households.index.isin(persons["household_id"])).sum()

0

In [18]:
stat = persons.groupby(["household_id", "relate"]).size().reset_index()

In [24]:
stat[stat[0] > 1]["relate"].unique()

array([ 2,  3,  7,  4, 11,  6,  5, 12, 10, 15,  8,  9, 14])

In [28]:
households[["age_of_head", "hh_age_of_head"]]

,age_of_head,hh_age_of_head
household_id,,
0,61,gt35-lt65
1,51,gt35-lt65
2,64,gt35-lt65
3,53,gt35-lt65
4,62,gt35-lt65
...,...,...
2608018,65,gt65
2608019,63,gt35-lt65
2608020,71,gt65


In [43]:
households.columns

Index(['serialno', 'cars', 'income', 'race_of_head', 'age_of_head', 'workers',
       'hispanic_status_of_head', 'tenure', 'recent_mover', 'sf_detached',
       'hh_age_of_head', 'hh_race_of_head', 'hispanic_head', 'hh_size',
       'hh_cars', 'hh_children', 'seniors', 'hh_income', 'hh_workers',
       'tenure_mover', 'hh_seniors', 'block_id', 'lcm_county_id', 'persons',
       'gt55', 'gt2', 'hh_type'],
      dtype='object')

In [59]:
import numpy as np

In [60]:
persons["child"] = np.where(persons["relate"].isin([2, 3, 4, 14]), 1, 0)
persons["senior"] = np.where(persons["age"]>=65, 1, 0)
persons["ind"] = 1


In [71]:
households_stats = persons.groupby(["household_id"]).agg(children = ("child", "sum"),
                                                            seniors = ("senior", "sum"),
                                                            size = ("ind", "sum"))
households_stats["hh_children"] = np.where(households_stats["children"]>0, "yes", "no")
households_stats["gt55"] = np.where(households_stats["seniors"]>0, 1, 0)
households_stats["hh_seniors"] = np.where(households_stats["seniors"]>0, "yes", "no")



In [88]:
persons[persons["household_id"]==2608018]

,age,earning,edu,hispanic,hours,member_id,race_id,relate,sex,student,work_at_home,worker,household_id,person_age,person_sex,race,hispanic.1,p_hispanic,MAR,child
person_id,,,,,,,,,,,,,,,,,,,,
6808549,62,14000.0,20.0,0,35.0,2,1,5,1,0,1,1,2608018,above 60,male,white,0,no,1,0
6808550,65,6400.0,21.0,0,30.0,1,1,0,2,0,1,1,2608018,above 60,female,white,0,no,1,0


In [90]:
orca.get_table("households").local[["gt55"]].value_counts()

gt55
0       1478484
1       1126902
dtype: int64

In [58]:
persons[persons["household_id"]==1225]

,age,earning,edu,hispanic,hours,member_id,race_id,relate,sex,student,work_at_home,worker,household_id,person_age,person_sex,race,hispanic.1,p_hispanic,MAR
person_id,,,,,,,,,,,,,,,,,,,
570,48,98000.0,23.0,0,55.0,1,1,0,2,0,0,1,1225,35 to 60,female,white,0,no,1
572,49,64000.0,18.0,0,40.0,2,1,1,1,0,0,1,1225,35 to 60,male,white,0,no,1
574,18,500.0,14.0,0,40.0,3,1,2,1,1,0,1,1225,19 and under,male,white,0,no,5


In [57]:
households.loc[1225]

serialno                     2012000089101
cars                                   1.0
income                            162500.0
race_of_head                             1
age_of_head                             48
workers                                3.0
hispanic_status_of_head                  0
tenure                                   2
recent_mover                             0
sf_detached                            yes
hh_age_of_head                   gt35-lt65
hh_race_of_head                      white
hispanic_head                           no
hh_size                              three
hh_cars                                one
hh_children                             no
seniors                                0.0
hh_income                            gt150
hh_workers                     two or more
tenure_mover               rent not recent
hh_seniors                              no
block_id                   060014001001046
lcm_county_id                        06001
persons    

In [50]:
households[households["hh_children"]=="yes"]

,serialno,cars,income,race_of_head,age_of_head,workers,hispanic_status_of_head,tenure,recent_mover,sf_detached,...,hh_income,hh_workers,tenure_mover,hh_seniors,block_id,lcm_county_id,persons,gt55,gt2,hh_type
household_id,,,,,,,,,,,,,,,,,,,,,
123,2010000916731,1.0,109500.0,1,53,0.0,0,1,0,yes,...,gt100-lt150,none,own not recent,no,060014001001062,06001,2,0,1,3
124,2013000806181,1.0,180000.0,2,56,1.0,0,1,0,yes,...,gt150,one,own not recent,no,060014001001054,06001,2,1,1,4
125,2009000752083,1.0,207000.0,1,54,2.0,0,1,0,yes,...,gt150,two or more,own not recent,no,060014001001036,06001,3,0,1,3
126,2010000320442,1.0,81000.0,1,50,0.0,0,1,0,no,...,gt60-lt100,none,own not recent,no,060014001001027,06001,2,0,1,3
127,2010000132926,1.0,82300.0,1,51,1.0,0,2,0,no,...,gt60-lt100,one,rent not recent,no,060014001001024,06001,2,0,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607934,2010000248119,2.0,71000.0,1,52,2.0,0,2,0,yes,...,gt60-lt100,two or more,rent not recent,no,060971543043013,06097,3,0,1,7
2607972,2012000276453,2.0,126000.0,1,41,2.0,0,1,0,yes,...,gt100-lt150,two or more,own not recent,no,060971543043038,06097,3,0,1,3
2607989,2010001316697,3.0,80100.0,1,30,2.0,0,2,0,yes,...,gt60-lt100,two or more,rent not recent,no,060971543043124,06097,3,0,1,7


In [45]:
households[["hh_age_of_head", "hh_children"]]

,hh_age_of_head,hh_children
household_id,,
0,gt35-lt65,no
1,gt35-lt65,no
2,gt35-lt65,no
3,gt35-lt65,no
4,gt35-lt65,no
...,...,...
2608018,gt65,no
2608019,gt35-lt65,no
2608020,gt65,no
